In [4]:
import torch as th
import numpy as np
from functions_ngates_v2 import calc_variance_pure, calc_variance_ng_cnot_batched, _kron_batched
from functions_ngates_v1 import calc_variance_ng_crx_batched

ModuleNotFoundError: No module named 'functions_ngates_v2'

In [16]:
Lx = 2
Ly = 5

matrix = np.zeros((Lx,Ly))

for i in range(Lx*Ly):
        print(i, i%Lx, i//Lx)

0 0 0
1 1 0
2 0 1
3 1 1
4 0 2
5 1 2
6 0 3
7 1 3
8 0 4
9 1 4


In [13]:
f = lambda x,y : (x+1)%Lx*Ly + (y+1)%Ly
f(0,0)

6

In [57]:
L = 1
N = 2

batch_size = 1

phi = th.randn(size=(batch_size, N-1, L), dtype=th.float64) * np.pi/4

cnot = th.zeros((batch_size, N-1, L, 4, 4), dtype=th.complex128,)
cnot[:,:,:,0,0] = 1
cnot[:,:,:,1,1] = 1
cnot[:,:,:,2,2] = (1 + th.exp(1j * 4 * phi))/2
cnot[:,:,:,3,3] = (1 + th.exp(1j * 4 * phi))/2
cnot[:,:,:,2,3] = (1 - th.exp(1j * 4 * phi))/2
cnot[:,:,:,3,2] = (1 - th.exp(1j * 4 * phi))/2

cnot

tensor([[[[[1.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j],
           [0.0000+0.0000j, 1.0000+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j],
           [0.0000+0.0000j, 0.0000+0.0000j, 0.0169+0.1290j, 0.9831-0.1290j],
           [0.0000+0.0000j, 0.0000+0.0000j, 0.9831-0.1290j, 0.0169+0.1290j]]]]],
       dtype=torch.complex128)

In [60]:
cnot[:,0,0] @ cnot[:,0,0].conj().transpose(-1,-2)

tensor([[[ 1.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
         [ 0.0000e+00+0.j,  1.0000e+00+0.j,  0.0000e+00+0.j,  0.0000e+00+0.j],
         [ 0.0000e+00+0.j,  0.0000e+00+0.j,  1.0000e+00+0.j, -4.8572e-17+0.j],
         [ 0.0000e+00+0.j,  0.0000e+00+0.j, -4.8572e-17+0.j,  1.0000e+00+0.j]]],
       dtype=torch.complex128)

In [22]:
cnot = th.zeros((batch_size, N-1, L, 4, 4), dtype=th.complex128,)
cnot[:,:,:,0,0] = 1
cnot[:,:,:,1,1] = 1
cnot[:,:,:,2,2] = (1 + th.exp(1j * 4 * phi))/2
cnot[:,:,:,3,3] = (1 + th.exp(1j * 4 * phi))/2
cnot[:,:,:,2,3] = (1 - th.exp(1j * 4 * phi))/2
cnot[:,:,:,3,2] = (1 - th.exp(1j * 4 * phi))/2
np.round(cnot.numpy())

array([[[[[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 1.+0.j, 0.-0.j],
          [0.+0.j, 0.+0.j, 0.-0.j, 1.+0.j]]]],



       [[[[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
          [0.+0.j, 0.+0.j, 0.+0.j, 1.-0.j],
          [0.+0.j, 0.+0.j, 1.-0.j, 0.+0.j]]]]])

In [4]:
calc_variance_ng_cnot_batched(4,10,n_sim=1000, n_sim_noise=500).shape

torch.Size([1000, 500])

In [6]:
calc_variance_ng_crx_batched(4,10,np.pi/2,n_sim=1000,n_sim_noise=500).shape

torch.Size([1000, 500])

In [ ]:
import torch.nn as nn

class circuit(nn.Module):
    def __init__(self, phi_init=phi):
        super().__init__()
        self.phi = nn.Parameter(torch.tensor(phi_init, dtype=torch.float64))

    def forward(self):
        # Example: apply Rx[0] and Ry[0] using the optimizable phi
        rx_mat = Rx[0](self.phi)
        ry_mat = Ry[0](self.phi)
        return rx_mat @ ry_mat

# Instantiate the module
opt_module = MyOptimModule()